In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import pdist, squareform


In [ ]:
fic_epita_kantar_codes = pd.read_csv('data/fic_epita_kantar_codes.csv', sep=';')
fic_epita_kantar_codes

In [ ]:
from utils.utils import select_features
columns = [
    'A9_1_slice', 'A9_2_slice', 'A9_3_slice', 'A9_4_slice', 'A9_5_slice', 
    'A9_6_slice', 'A9_7_slice', 'A9_8_slice', 'A9_9_slice', 'A9_10_slice', 
    'A9_11_slice', 'A9_12_slice', 'A9_13_slice', 'A9_14_slice', 'A9_15_slice', 
    'A9_16_slice', 'A10_1_slice', 'A10_2_slice', 'A10_3_slice', 'A10_4_slice', 
    'A10_5_slice', 'A10_6_slice', 'A10_7_slice', 'A10_8_slice', 'A11_1_slice', 
    'A11_2_slice', 'A11_3_slice', 'A11_4_slice', 'A11_5_slice', 'A11_6_slice', 
    'A11_7_slice', 'A11_8_slice', 'A11_9_slice', 'A11_10_slice', 'A11_11_slice', 
    'A11_12_slice', 'A11_13_slice'
]

# Combine these columns into a feature space
features = select_features(fic_epita_kantar_codes, columns)
features = features.values
features

In [ ]:
weights = fic_epita_kantar_codes['weight'].values

len(weights)

In [ ]:
# Standardize the feature space
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

features_scaled[0]

In [ ]:
inertia = []
k_range = range(2, 11)  # Essayer de 1 à 10 clusters

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(features_scaled, sample_weight=weights)
    inertia.append(kmeans.inertia_)

# Affichage du graphique de la méthode du coude
plt.figure(figsize=(8, 5))
plt.plot(k_range, inertia, marker='o')
plt.title("Elbow Method for Optimal k in K-means")
plt.xlabel("Number of Clusters (k)")
plt.ylabel("Inertia")
plt.xticks(k_range)
plt.show()

In [ ]:
from utils.utils import weighted_distance_matrix

weighted_distances = weighted_distance_matrix(features_scaled, weights)

# Conversion en format condensé pour linkage
condensed_weighted_distances = squareform(weighted_distances)

# Clustering hiérarchique avec la matrice de distances pondérées
Z = linkage(condensed_weighted_distances, method='ward')
Z

In [ ]:
plt.figure(figsize=(25, 10))
dendrogram(Z, orientation='top')
plt.show()